<a href="https://colab.research.google.com/github/dh610/Advanced-AI/blob/main/Restormer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os

sys.path.append("/content/drive/MyDrive/Advanced-AI/InstructIR/")
os.chdir("/content/drive/MyDrive/Advanced-AI/InstructIR/")
!ls

app.py	  datasets.py	      index.html  predict.py		   results    text
cog.yaml  demo.ipynb	      LICENSE	  __pycache__		   static     utils.py
configs   eval_instructir.py  metrics.py  README.md		   test-data
data	  images	      models	  requirements_gradio.txt  test.py


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
import torch

from utils import *
from models import instructir

from text.models import LanguageModel, LMHead

SEED=42
seed_everything(SEED=SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
CONFIG     = "configs/eval5d.yml"
LM_MODEL   = "models/lm_instructir-7d.pt"
MODEL_NAME = "models/im_instructir-7d.pt"

# parse config file
with open(os.path.join(CONFIG), "r") as f:
    config = yaml.safe_load(f)

cfg = dict2namespace(config)

In [ ]:
print ("Creating InstructIR")
model = instructir.create_model(input_channels =cfg.model.in_ch, width=cfg.model.width, enc_blks = cfg.model.enc_blks,
                            middle_blk_num = cfg.model.middle_blk_num, dec_blks = cfg.model.dec_blks, txtdim=cfg.model.textdim)

print (device)
model = model.to(device)

# LOAD IMAGE MODEL

Creating InstructIR
cuda


In [ ]:
# LANGUAGE MODEL

if cfg.model.use_text:
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Initialize the LanguageModel class
    LMODEL = cfg.llm.model
    language_model = LanguageModel(model=LMODEL)
    lm_head = LMHead(embedding_dim=cfg.llm.model_dim, hidden_dim=cfg.llm.embd_dim, num_classes=cfg.llm.nclasses)
    lm_head = lm_head.to(device)
    lm_nparams   = count_params (lm_head)

    print ("LMHEAD MODEL CKPT:", LM_MODEL)
    lm_head.load_state_dict(torch.load(LM_MODEL, map_location=torch.device('cpu')), strict=True)
    print ("Loaded weights!")

else:
    LMODEL = None
    language_model = None
    lm_head = None
    lm_nparams = 0

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/34.8M [00:00<?, ?B/s]

LMHEAD MODEL CKPT: models/lm_instructir-7d.pt


<ipython-input-6-8489ccebc86c>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lm_head.load_state_dict(torch.load(LM_MODEL, map_location=torch.device('cpu')), strict=True)

Loaded weights!


In [ ]:
lm_embd = language_model("I want my photo to be clear, not rainy").to(device)
text_embd, deg_pred = lm_head (lm_embd)

torch.argmax(deg_pred)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor(2, device='cuda:0')

In [ ]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn

input_dir = '/content/drive/MyDrive/Advanced-AI/InstructIR/test-data/Rain100L/input'
target_dir = '/content/drive/MyDrive/Advanced-AI/InstructIR/test-data/Rain100L/target'

class ImagePairDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(input_dir) if f.endswith('.png')]  # 이미지 파일 이름 리스트

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.image_filenames[idx])
        input_image = Image.open(input_image_path).convert("RGB")

        target_image_path = os.path.join(self.target_dir, self.image_filenames[idx])
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = ImagePairDataset(input_dir=input_dir, target_dir=target_dir, transform=transform)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



InstructIR(
  (intro): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ending): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoders): ModuleList(
    (0): Sequential(
      (0): NAFBlock(
        (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        (conv3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (sca): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (sg): SimpleGate()
        (conv4): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv5): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (norm1): LayerNorm2d()
        (norm2): LayerNorm2d()
        (dropout1): Identity()
        (dropout2): Identity()
      )
      (1): NAFBlock(
        (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        

In [ ]:
from tqdm import tqdm

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for input_images, target_images in tqdm(train_loader):
        input_images, target_images = input_images.to(device), target_images.to(device)

        optimizer.zero_grad()

        outputs = model(input_images, text_embd)
        loss = criterion(outputs, target_images)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

 17%|█▋        | 1/6 [00:19<01:35, 19.13s/it]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:

model.eval()
test_loss = 0.0
with torch.no_grad():
    for input_images, target_images in test_loader:
        input_images, target_images = input_images.to(device), target_images.to(device)

        outputs = model(input_images, text_embd)
        loss = criterion(outputs, target_images)
        test_loss += loss.item()

print(f"Test Loss: {test_loss / len(test_loader)}")